Connect to the (locally stored) SQLite database

In [1]:
import pandas as pd
import sqlite3
con = sqlite3.connect('sqlite_db_pythonsqlite.db')

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [2]:
q10 = pd.read_sql_query('SELECT * FROM'
'(SELECT `f`.`name` AS `facility`,'
'SUM(CASE WHEN `b`.`memid` = 0 THEN `f`.`guestcost`*`b`.`slots` ELSE `f`.`membercost`*`b`.`slots` END) AS `revenue`'
'FROM `Bookings`  AS `b`'
'LEFT JOIN `Facilities` AS `f`'
'ON `f`.`facid` = `b`.`facid`'
'LEFT JOIN `Members` AS `m`'
'ON `b`.`memid` = `m`.`memid`'
'GROUP BY `facility`'
'ORDER BY `revenue`) AS `q`'
'WHERE `revenue` < 1000.0', con)
print(q10)

        facility  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


Q11: Produce a report of members and who recommended them in alphabetic surname, firstname order

In [3]:
q11 = pd.read_sql_query('SELECT `m`.`surname` AS `member surname`, `m`.`firstname` AS `member firstname`,'
'`q`.`surname` AS `recommended`, `q`.`firstname` AS `by`'
'FROM `Members` AS `m`'
'LEFT JOIN (SELECT `memid`, `firstname`, `surname` FROM `Members`) AS `q`'
'ON `m`.`recommendedby` = `q`.`memid`'
'WHERE `q`.`memid` != 0', con)
print(q11)

       member surname member firstname recommended         by
0            Joplette           Janice       Smith     Darren
1             Butters           Gerald       Smith     Darren
2                Dare            Nancy    Joplette     Janice
3              Boothe              Tim      Rownam        Tim
4            Stibbons           Ponder       Tracy     Burton
5                Owen          Charles       Smith     Darren
6               Jones            David    Joplette     Janice
7               Baker             Anne    Stibbons     Ponder
8               Smith             Jack       Smith     Darren
9               Bader         Florence    Stibbons     Ponder
10              Baker          Timothy     Farrell     Jemima
11             Pinker            David     Farrell     Jemima
12            Genting          Matthew     Butters     Gerald
13          Mackenzie             Anna       Smith     Darren
14             Coplin             Joan       Baker    Timothy
15      

Q12: Find the facilities with their usage by member, but not guests

In [4]:
q12 = pd.read_sql_query('SELECT `facility`, `firstname`, `surname`, `usage` FROM'
'(SELECT `f`.`name` AS `facility`,`m`.`memid`,`m`.`firstname`, `m`.`surname`,'
'SUM(`b`.`slots`) AS `usage`'
'FROM `Bookings`AS `b`'
'LEFT JOIN `Facilities` AS `f`'
'ON `f`.`facid` = `b`.`facid`'
'LEFT JOIN `Members` AS `m`'
'ON `b`.`memid` = `m`.`memid`'
'GROUP BY `m`.`memid`'
'ORDER BY `f`.`facid`)'
'WHERE `memid` != 0', con)
print(q12)

           facility  firstname            surname  usage
0    Tennis Court 1      Tracy              Smith    435
1    Tennis Court 1     Gerald            Butters    409
2    Tennis Court 1    Charles               Owen    345
3    Tennis Court 1       Anne              Baker    296
4    Tennis Court 1      David            Farrell     50
5    Tennis Court 1       John               Hunt     40
6    Tennis Court 2     Burton              Tracy    366
7    Tennis Court 2        Tim             Boothe    440
8    Tennis Court 2     Ponder           Stibbons    249
9    Tennis Court 2      David              Jones    305
10   Tennis Court 2    Timothy              Baker    290
11   Tennis Court 2  Ramnaresh             Sarwin    153
12  Badminton Court      Nancy               Dare    267
13  Badminton Court   Florence              Bader    237
14  Badminton Court       Anna          Mackenzie    231
15  Badminton Court    Douglas              Jones     37
16  Badminton Court      Henry 

Q13: Find the facilities usage by month, but not guests

In [5]:
q13 = pd.read_sql_query(''
'SELECT `f`.`name` AS `facility`,'
'SUBSTR(`b`.`starttime`,1,7) AS `month`,'
'SUM(CASE WHEN `b`.`memid` = 0 THEN 0 ELSE `b`.`slots` END) AS `usage`'
'FROM `Bookings` AS `b`'
'LEFT JOIN `Facilities` AS `f`'
'ON `f`.`facid` = `b`.`facid`'
'LEFT JOIN `Members` AS `m`'
'ON `b`.`memid` = `m`.`memid`'
'GROUP BY `f`.`facid`,`month`'
'ORDER BY `f`.`facid`,`month`', con)
print(q13)

           facility    month  usage
0    Tennis Court 1  2012-07    201
1    Tennis Court 1  2012-08    339
2    Tennis Court 1  2012-09    417
3    Tennis Court 2  2012-07    123
4    Tennis Court 2  2012-08    345
5    Tennis Court 2  2012-09    414
6   Badminton Court  2012-07    165
7   Badminton Court  2012-08    414
8   Badminton Court  2012-09    507
9      Table Tennis  2012-07     98
10     Table Tennis  2012-08    296
11     Table Tennis  2012-09    400
12   Massage Room 1  2012-07    166
13   Massage Room 1  2012-08    316
14   Massage Room 1  2012-09    402
15   Massage Room 2  2012-07      8
16   Massage Room 2  2012-08     18
17   Massage Room 2  2012-09     28
18     Squash Court  2012-07     50
19     Squash Court  2012-08    184
20     Squash Court  2012-09    184
21    Snooker Table  2012-07    140
22    Snooker Table  2012-08    316
23    Snooker Table  2012-09    404
24       Pool Table  2012-07    110
25       Pool Table  2012-08    303
26       Pool Table  2012-09

Close the connection since we're done

In [6]:
con.close()